In [1]:
import sys

print(sys.path)
sys.path.append("/home/momentino/GarbageDetectionAndSegmentation")

['/home/momentino/GarbageDetectionAndSegmentation/notebooks', '/usr/lib64/python310.zip', '/usr/lib64/python3.10', '/usr/lib64/python3.10/lib-dynload', '', '/home/momentino/.local/lib/python3.10/site-packages', '/usr/lib64/python3.10/site-packages', '/usr/lib/python3.10/site-packages']


In [2]:
!pip install sklearn

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install scikit-image

Defaulting to user installation because normal site-packages is not writeable


In [4]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from skimage.feature import hog
from imageio import imread
import joblib

import random as rand
import numpy as np 
import cv2
import glob
import time

import matplotlib.pyplot as plt
%matplotlib inline

from utils import convert, show_images
from featuresourcer import CannyFeatureExtractor

In [5]:
sourcer_params = {
  'color_model': 'hls',                # hls, hsv, yuv, ycrcb
  'first_thresh': 10,             #
  'second_thresh' : 200,
}

start_frame = imread("/home/momentino/Scaricati/archive/garbage classification/Garbage classification/cardboard/cardboard4.jpg")
sourcer = CannyFeatureExtractor(sourcer_params, start_frame)

/tmp/ipykernel_28121/2347638959.py:7: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  start_frame = imread("/home/momentino/Scaricati/archive/garbage classification/Garbage classification/cardboard/cardboard4.jpg")


In [6]:
print("Loading images to memory...")
t_start = time.time()

garbage_imgs, nongarbage_imgs = [], []
garbage_paths = glob.glob('/home/momentino/Scaricati/archive/garbage classification/Garbage classification/*/*.jpg')
nongarbage_paths = glob.glob('/home/momentino/Scaricati/archive/garbage classification/negative/*.jpg')

for path in garbage_paths: garbage_imgs.append(imread(path))
for path in nongarbage_paths: nongarbage_imgs.append(imread(path))

garbage_imgs, nongarbage_imgs = np.asarray(garbage_imgs), np.asarray(nongarbage_imgs)
total_garbage, total_nongarbage = garbage_imgs.shape[0], nongarbage_imgs.shape[0]

print("... Done")
print("Time Taken:", np.round(time.time() - t_start, 2))
print("Garbage images shape: ", garbage_imgs.shape)
print("Non-garbage images shape: ", nongarbage_imgs.shape)

print("Extracting features... This might take a while...")
t_start = time.time()

garbage_features, nongarbage_features = [], []

print("Garbage...")
for i,img in enumerate(garbage_imgs):
  #print(garbage_paths[i])
  garbage_features.append(sourcer.features(img))
  print('█', end = '')

print()
print("Non-Garbage...")
for img in nongarbage_imgs:
  nongarbage_features.append(sourcer.features(img))
  print('█', end = '')
                         
garbage_features = np.asarray(garbage_features)
nongarbage_features = np.asarray(nongarbage_features)

print()
print("...Done")
print("Time Taken:", np.round(time.time() - t_start, 2))
print("Garbage features shape: ", garbage_features.shape)
print("Non-Garbage features shape: ", nongarbage_features.shape)

print("Scaling features...")
t_start = time.time()
print(" GARBAGE SHAPE ",garbage_features.dtype)
print(" NONGARBAGE SHAPE ", nongarbage_features.dtype)
unscaled_x = np.vstack((garbage_features, nongarbage_features)).astype(np.float64)
scaler = StandardScaler().fit(unscaled_x)
x = scaler.transform(unscaled_x)
y = np.hstack((np.ones(total_garbage), np.zeros(total_nongarbage)))

print("...Done")
print("Time Taken:", np.round(time.time() - t_start, 2))
print(" x shape: ", x.shape, " y shape: ", y.shape)

print("Training classifier...")
t_start = time.time()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2,
                                                    random_state = rand.randint(1, 100))
svc = LinearSVC()
svc.fit(x_train, y_train)
accuracy = svc.score(x_test, y_test)

print("...Done")
print("Time Taken:", np.round(time.time() - t_start, 2))
print("Accuracy: ", np.round(accuracy, 4))

/tmp/ipykernel_28121/471062377.py:8: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  for path in garbage_paths: garbage_imgs.append(imread(path))


Loading images to memory...


/tmp/ipykernel_28121/471062377.py:9: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  for path in nongarbage_paths: nongarbage_imgs.append(imread(path))


... Done
Time Taken: 1.17
Garbage images shape:  (2275, 48, 64, 3)
Non-garbage images shape:  (1068, 48, 64, 3)
Extracting features... This might take a while...
Garbage...
███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

/home/momentino/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [7]:
print ("Saving models...")

joblib.dump(svc, 'svc.pkl')
joblib.dump(scaler, 'scaler.pkl')

print("...Done")

Saving models...
...Done


In [8]:
print ("Loading models...")

svc = joblib.load('svc.pkl')
scaler = joblib.load('scaler.pkl')

print("...Done")

Loading models...
...Done


In [9]:
sourcer = FeatureSourcer(sourcer_params, start_frame)

f1 = sourcer.features(nongarbage_imgs[rand.randint(0, total_nongarbage)])
f2 = sourcer.features(garbage_imgs[rand.randint(0, total_garbage)])
f3 = sourcer.features(garbage_imgs[rand.randint(0, total_garbage)])
f4 = sourcer.features(nongarbage_imgs[rand.randint(0, total_nongarbage)])

f = scaler.transform([f1, f2, f3, f4])
print(svc.predict(f))

NameError: name 'FeatureSourcer' is not defined